In [1]:
# If running this notebook without having carla installed
%cd ..\

c:\Users\Surfer\Desktop\EaFML Project\CARLA


In [2]:
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [3]:
import carla
from carla import logging
logger = carla.get_logger("carla")
logger.setLevel(logging.WARNING)

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [4]:


from carla.data.catalog import OnlineCatalog
import carla.evaluation.catalog as evaluation_catalog
from carla.models.catalog import MLModelCatalog
import carla.recourse_methods.catalog as recourse_catalog
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.processing import reconstruct_encoding_constraints
from carla import Benchmark
from carla.recourse_methods.catalog.arar import ARAR
from carla.models import MLModel

import torch
import numpy as np
import pandas as pd


### Example: Adult dataset with ANN model and ARAR recourse method

In [5]:
# load dataset
data_name = "adult"
dataset = OnlineCatalog(data_name)

In [6]:
# load catalog model
model_type = "ann"
ml_model = MLModelCatalog(
    dataset,
    model_type=model_type,
    load_online=True,
    backend="pytorch"
)

In [7]:
# using the standard hyperparameters
hyperparams = None
recourse_method = recourse_catalog.ARAR(ml_model, hyperparams)

In [8]:
factuals = predict_negative_instances(ml_model, dataset.df)
factuals = factuals

counterfactuals = recourse_method.get_counterfactuals(factuals)
counterfactuals = counterfactuals.dropna()

Pct left: 0.821 Lambda: 0.0000: 100%|██████████| 100/100 [01:49<00:00,  1.10s/it]


In [9]:
counterfactuals

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
4,0.578932,0.231562,0.633211,0.099966,0.099824,...,0.0,0.0,1.0,1.0,1.0
7,0.414448,0.171745,0.766521,0.099960,0.099790,...,1.0,0.0,0.0,1.0,1.0
15,0.455626,0.168843,0.499874,0.099965,0.568596,...,1.0,0.0,1.0,1.0,1.0
18,0.606340,0.211692,0.699880,0.099967,0.099828,...,1.0,0.0,0.0,1.0,0.0
31,0.195398,0.206866,0.899884,0.099968,0.099833,...,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
40277,0.304893,0.254743,0.499862,0.099962,0.483640,...,1.0,0.0,1.0,1.0,1.0
40284,0.578920,0.251961,0.633190,0.099961,0.099776,...,0.0,1.0,1.0,0.0,0.0
40302,0.537853,0.141497,0.633197,0.099964,0.099789,...,0.0,0.0,1.0,1.0,0.0
40316,0.524090,0.282756,0.633200,0.099963,0.099808,...,0.0,0.0,1.0,1.0,1.0


### Example: With hyperparameters

In [10]:
hyperparams = {'lr': 0.01, 'epsilon': 0.05, 'outer_iters': 10, 'inner_iters': 50, 'inner_max_pgd':True}
recourse_method = recourse_catalog.ARAR(ml_model, hyperparams)

In [11]:
factuals = predict_negative_instances(ml_model, dataset.df)
factuals = factuals[:50]

counterfactuals = recourse_method.get_counterfactuals(factuals)
counterfactuals = counterfactuals.dropna()

Pct left: 0.780 Lambda: 0.3487: 100%|██████████| 10/10 [00:16<00:00,  1.64s/it]


# Experiments

## Comparing ARAR and Wachter: generating attacks on the counterfactuals in the epsilon ball via gradient

In [ ]:
def experiment_generate_perturbations_gradient(model, counterfactuals, y_target = 1, epsilon = 0.05, binary_cat_features = True):
    """ Generate adversarial perturbations on the data based on the gradient """

    factuals = model.get_ordered_features(counterfactuals)

    encoded_feature_names = model.data.encoder.get_feature_names(
        model.data.categorical
    )
    cat_features_indices = [
        factuals.columns.get_loc(feature) for feature in encoded_feature_names
    ]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    x = torch.from_numpy(factuals.to_numpy().astype(np.float32)).to(device)
    preds = model.predict(x)
    #now pertubate the data
    D = x.shape[1]
    loss = torch.nn.BCEWithLogitsLoss(reduction='none')
    target_vec = torch.ones(x.shape[0], device=device) * y_target
    x_pertb = torch.autograd.Variable(torch.zeros(x.shape, device =device), requires_grad=True)
    x_adv = x + x_pertb
    loss_x = torch.mean(loss(model.predict(x_adv).squeeze(), target_vec))
    grad = torch.autograd.grad(loss_x, x_pertb, create_graph=False)[0]
    sum = torch.sum(grad, dim=-1)
    pertb = grad
    pertb[sum!=0] = pertb [sum!=0]/ torch.linalg.norm(grad[sum!=0], dim=-1, keepdims=True) * epsilon
    x_pertb = x + pertb
    x_pertb = reconstruct_encoding_constraints(x_pertb, cat_features_indices, binary_cat_features)
    preds_pertb = model.predict(x_pertb)
    return x_pertb.cpu().detach().numpy(), preds.cpu().detach().numpy(), preds_pertb.cpu().detach().numpy()

## Comparing ARAR and Wachter: generating random samples in the epsilon ball and classifying them

In [ ]:
def experiment_generate_perturbations_random(model, counterfactuals, n_samples = 20, epsilon = 0.05, binary_cat_features = True):
    """ Generates random sampels in the epsilon ball"""
    factuals = model.get_ordered_features(counterfactuals)
        
    encoded_feature_names = model.data.encoder.get_feature_names(
        model.data.categorical
    )
    cat_features_indices = [
        factuals.columns.get_loc(feature) for feature in encoded_feature_names
    ]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    x = torch.from_numpy(factuals.to_numpy().astype(np.float32)).to(device)
    pred_pertb_list = torch.zeros((n_samples, x.shape[0]), device=device)
    for i in range(n_samples):
        pertb = torch.randn(x.shape, device=device)
        sum = torch.sum(pertb, dim=-1)
        pertb[sum!=0] = pertb [sum!=0]/ torch.linalg.norm(pertb[sum!=0], dim=-1, keepdims=True) * epsilon
        x_pertb = x + pertb
        x_pertb = reconstruct_encoding_constraints(x_pertb, cat_features_indices, binary_cat_features)
        preds_pertb = model.predict(x_pertb).squeeze()
        pred_pertb_list[i] = preds_pertb>=0.5
    pred_pertb_list = pred_pertb_list.cpu().numpy()
    return np.mean(pred_pertb_list, axis=0)

### Concise Experiments

### All Datasets

In [14]:
EPSILON_TEST = 0.1


# Hyperparams ARAR
LR = 0.1
OUTER_ITERS = 100
INNER_ITERS = 50


num_test_factuals = 100000
num_test_factuals_wachter = 500 # Wachter is much slower due to missing parallelism 
for model_type in ["ann", "linear"]:
    print("#################################################################################################################")
    print("######### model_type: ", model_type)
    print("#################################################################################################################")
    for data_name in ["adult", "compas", "give_me_some_credit", "heloc"]: # "adult"
        print("###### Dataset Name: ", data_name)
        data = OnlineCatalog(data_name)

        model = MLModelCatalog(data, model_type, backend="pytorch")
        # get factuals
        factuals = predict_negative_instances(model, data.df)
        factuals = factuals.sample(frac=1, random_state=8912983) # Shuffle data for good measure (only needed for wachter, as we do not use the whole dataset)
        test_factual = factuals.iloc[:num_test_factuals]

        if len(test_factual) == 0:
            print("No test factuals classified as the negative instance were found -> skipping this evaluation")
            continue

        # Generate Counterfactuals
        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.1}
        cfs_arar_eps01 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'inner_max_pgd': True, 'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.1}
        cfs_arar_eps01_pgd = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.05}
        cfs_arar_eps005 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'inner_max_pgd': True, 'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.05}
        cfs_arar_eps005_pgd = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.0}
        cfs_arar_eps0 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()
        
        # ARAR with hyperparameters to match the ones that are used in Wachter -> obtains very similar results
        hyperparams = {'y_target':1, 'binary_cat_features': True, 'epsilon':0.0,
                        'lr': 0.01, # smaller learning rate
                        'lambd_init': 0.0, # no l1-regularization on the delta
                        'decay_rate': 1.0, 
                        'outer_iters': OUTER_ITERS, # 1000
                        'inner_iters': INNER_ITERS,
                        }
        cfs_arar_eps0_like_wachter = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()


        hyperparams = {"loss_type": "BCE", "binary_cat_features": True, "t_max_min": 0.01, "verbose_logging": False}
        cfs_wachter = recourse_catalog.Wachter(model, hyperparams).get_counterfactuals(test_factual[:num_test_factuals_wachter]).dropna()

        print("### Number of counterfactuals found / Total number of factuals")
        print("ARAR, eps=0.1: ", len(cfs_arar_eps01), " / ", len(test_factual), " = ", len(cfs_arar_eps01) / len(test_factual))
        print("ARAR, eps=0.1, pgd: ", len(cfs_arar_eps01_pgd), " / ", len(test_factual), " = ", len(cfs_arar_eps01_pgd) / len(test_factual))
        print("ARAR, eps=0.05: ", len(cfs_arar_eps005), " / ", len(test_factual), " = ", len(cfs_arar_eps005) / len(test_factual))
        print("ARAR, eps=0.05, pgd: ", len(cfs_arar_eps005_pgd), " / ", len(test_factual), " = ", len(cfs_arar_eps005_pgd) / len(test_factual))
        print("ARAR, eps=0.0: ", len(cfs_arar_eps0), " / ", len(test_factual), " = ", len(cfs_arar_eps0) / len(test_factual))
        print("ARAR, eps=0.0, like Wachter: ", len(cfs_arar_eps0_like_wachter), " / ", len(test_factual), " = ", len(cfs_arar_eps0_like_wachter) / len(test_factual))
        print("Wachter: ", len(cfs_wachter), " / ", len(test_factual[:num_test_factuals_wachter]), " = ", len(cfs_wachter) / len(test_factual[:num_test_factuals_wachter]))

        
        # Generate perturbations with gradient
        print("### predictions classified as target / predictions classified as target after perturbation with gradient")
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps01, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps01_pgd, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1, pgd: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps005, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps005_pgd, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05, pgd: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps0, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps0_like_wachter, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0, like Wachter: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_wachter, epsilon=EPSILON_TEST)
        print("Wachter: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())

        # generate random perturbations
        print("### Proportion of classifications to target with random perturbations in epsilon ball: ")
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps01, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps01_pgd, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1, pgd: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps005, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps005_pgd, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05, pgd: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps0, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps0_like_wachter, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0, like Wachter: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_wachter, n_samples=1000, epsilon=EPSILON_TEST)
        print("Wachter: ", proportions.mean())



#################################################################################################################
######### model_type:  ann
#################################################################################################################
###### Dataset Name:  adult


100%|██████████| 500/500 [15:14<00:00,  1.83s/it]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  277  /  40325  =  0.006869187848729076
ARAR, eps=0.1, pgd:  277  /  40325  =  0.006869187848729076
ARAR, eps=0.05:  7216  /  40325  =  0.17894606323620582
ARAR, eps=0.05, pgd:  7216  /  40325  =  0.17894606323620582
ARAR, eps=0.0:  15621  /  40325  =  0.3873775573465592
ARAR, eps=0.0, like Wachter:  15621  /  40325  =  0.3873775573465592
Wachter:  203  /  500  =  0.406
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.9552383592017738
ARAR, eps=0.05, pgd:  1.0 0.9552383592017738
ARAR, eps=0.0:  1.0 0.5016964342871775
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9999962
ARAR, eps=0.05, pgd:  0.999998
ARAR, eps=0.0:  0.99

100%|██████████| 500/500 [07:07<00:00,  1.17it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  4  /  800  =  0.005
ARAR, eps=0.1, pgd:  4  /  800  =  0.005
ARAR, eps=0.05:  318  /  800  =  0.3975
ARAR, eps=0.05, pgd:  318  /  800  =  0.3975
ARAR, eps=0.0:  526  /  800  =  0.6575
ARAR, eps=0.0, like Wachter:  526  /  800  =  0.6575
Wachter:  322  /  500  =  0.644
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 1.0
ARAR, eps=0.05, pgd:  1.0 1.0
ARAR, eps=0.0:  1.0 0.8992395437262357
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  1.0
ARAR, eps=0.05, pgd:  1.0
ARAR, eps=0.0:  0.9997642
ARAR, eps=0.0, like Wachter:  0.59002095
Wachter:  0.5917857
###### Dataset Name:  give_me_some_credit
No test factuals classified as the

...

### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  317  /  317  =  1.0
ARAR, eps=0.1, pgd:  317  /  317  =  1.0
ARAR, eps=0.05:  317  /  317  =  1.0
ARAR, eps=0.05, pgd:  317  /  317  =  1.0
ARAR, eps=0.0:  317  /  317  =  1.0
ARAR, eps=0.0, like Wachter:  317  /  317  =  1.0
Wachter:  317  /  317  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.5772870662460567
ARAR, eps=0.05, pgd:  1.0 0.5772870662460567
ARAR, eps=0.0:  1.0 0.5772870662460567
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.972735
ARAR, eps=0.05, pgd:  0.9728392
ARAR, eps=0.0:  0.8986467
ARAR, eps=0.0, like Wachter:  0.5940915
Wachter:  0.59408516
###### Dataset Name:  give_me_some_credit


100%|██████████| 500/500 [00:04<00:00, 119.32it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  909  /  909  =  1.0
ARAR, eps=0.1, pgd:  909  /  909  =  1.0
ARAR, eps=0.05:  909  /  909  =  1.0
ARAR, eps=0.05, pgd:  909  /  909  =  1.0
ARAR, eps=0.0:  909  /  909  =  1.0
ARAR, eps=0.0, like Wachter:  909  /  909  =  1.0
Wachter:  500  /  500  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.7546754675467546
ARAR, eps=0.05, pgd:  1.0 0.7535753575357536
ARAR, eps=0.0:  1.0 0.7392739273927392
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9965764
ARAR, eps=0.05, pgd:  0.99677
ARAR, eps=0.0:  0.9589967
ARAR, eps=0.0, like Wachter:  0.64025635
Wachter:  0.64366394
###### Dataset Name:  heloc


100%|██████████| 500/500 [00:03<00:00, 137.72it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  5094  /  5094  =  1.0
ARAR, eps=0.1, pgd:  5094  /  5094  =  1.0
ARAR, eps=0.05:  5094  /  5094  =  1.0
ARAR, eps=0.05, pgd:  5094  /  5094  =  1.0
ARAR, eps=0.0:  5094  /  5094  =  1.0
ARAR, eps=0.0, like Wachter:  5094  /  5094  =  1.0
Wachter:  500  /  500  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 0.9998036906164115
ARAR, eps=0.05:  1.0 0.9167648213584609
ARAR, eps=0.05, pgd:  1.0 0.9155869650569297
ARAR, eps=0.0:  1.0 0.9110718492343934
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.99993896
ARAR, eps=0.05, pgd:  0.9999393
ARAR, eps=0.0:  0.9951147
ARAR, eps=0.0, like Wachter:  0.78642267
Wachter:  0.782816


#### Smaller LR, all else equal

In [15]:
EPSILON_TEST = 0.1


# Hyperparams ARAR
LR = 0.01
OUTER_ITERS = 100
INNER_ITERS = 50


num_test_factuals = 100000
num_test_factuals_wachter = 500 # Wachter is much slower due to missing parallelism 
for model_type in ["ann", "linear"]:
    print("#################################################################################################################")
    print("######### model_type: ", model_type)
    print("#################################################################################################################")
    for data_name in ["adult", "compas", "give_me_some_credit", "heloc"]: # "adult"
        print("###### Dataset Name: ", data_name)
        data = OnlineCatalog(data_name)

        model = MLModelCatalog(data, model_type, backend="pytorch")
        # get factuals
        factuals = predict_negative_instances(model, data.df)
        factuals = factuals.sample(frac=1, random_state=8912983) # Shuffle data for good measure (only needed for wachter, as we do not use the whole dataset)
        test_factual = factuals.iloc[:num_test_factuals]

        if len(test_factual) == 0:
            print("No test factuals classified as the negative instance were found -> skipping this evaluation")
            continue

        # Generate Counterfactuals
        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.1}
        cfs_arar_eps01 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'inner_max_pgd': True, 'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.1}
        cfs_arar_eps01_pgd = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.05}
        cfs_arar_eps005 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'inner_max_pgd': True, 'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.05}
        cfs_arar_eps005_pgd = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()

        hyperparams = {'lr': LR, 'outer_iters': OUTER_ITERS, 'inner_iters': INNER_ITERS, 'lambd_init': 1.0, 'decay_rate': 0.9, 'y_target':1, 'binary_cat_features': True, 'epsilon':0.0}
        cfs_arar_eps0 = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()
        
        # ARAR with hyperparameters to match the ones that are used in Wachter -> obtains very similar results
        hyperparams = {'y_target':1, 'binary_cat_features': True, 'epsilon':0.0,
                        'lr': 0.01, # smaller learning rate
                        'lambd_init': 0.0, # no l1-regularization on the delta
                        'decay_rate': 1.0, 
                        'outer_iters': OUTER_ITERS, # 1000
                        'inner_iters': INNER_ITERS,
                        }
        cfs_arar_eps0_like_wachter = ARAR(model, hyperparams).get_counterfactuals(test_factual).dropna()


        hyperparams = {"loss_type": "BCE", "binary_cat_features": True, "t_max_min": 0.01, "verbose_logging": False}
        cfs_wachter = recourse_catalog.Wachter(model, hyperparams).get_counterfactuals(test_factual[:num_test_factuals_wachter]).dropna()

        print("### Number of counterfactuals found / Total number of factuals")
        print("ARAR, eps=0.1: ", len(cfs_arar_eps01), " / ", len(test_factual), " = ", len(cfs_arar_eps01) / len(test_factual))
        print("ARAR, eps=0.1, pgd: ", len(cfs_arar_eps01_pgd), " / ", len(test_factual), " = ", len(cfs_arar_eps01_pgd) / len(test_factual))
        print("ARAR, eps=0.05: ", len(cfs_arar_eps005), " / ", len(test_factual), " = ", len(cfs_arar_eps005) / len(test_factual))
        print("ARAR, eps=0.05, pgd: ", len(cfs_arar_eps005_pgd), " / ", len(test_factual), " = ", len(cfs_arar_eps005_pgd) / len(test_factual))
        print("ARAR, eps=0.0: ", len(cfs_arar_eps0), " / ", len(test_factual), " = ", len(cfs_arar_eps0) / len(test_factual))
        print("ARAR, eps=0.0, like Wachter: ", len(cfs_arar_eps0_like_wachter), " / ", len(test_factual), " = ", len(cfs_arar_eps0_like_wachter) / len(test_factual))
        print("Wachter: ", len(cfs_wachter), " / ", len(test_factual[:num_test_factuals_wachter]), " = ", len(cfs_wachter) / len(test_factual[:num_test_factuals_wachter]))

        
        # Generate perturbations with gradient
        print("### predictions classified as target / predictions classified as target after perturbation with gradient")
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps01, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps01_pgd, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1, pgd: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps005, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps005_pgd, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05, pgd: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps0, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_arar_eps0_like_wachter, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0, like Wachter: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())
        x_pertb, preds, preds_pertb = experiment_generate_perturbations_gradient(model, cfs_wachter, epsilon=EPSILON_TEST)
        print("Wachter: ", (preds >= 0.5).mean(), (preds_pertb >= 0.5).mean())

        # generate random perturbations
        print("### Proportion of classifications to target with random perturbations in epsilon ball: ")
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps01, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps01_pgd, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.1, pgd: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps005, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps005_pgd, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.05, pgd: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps0, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_arar_eps0_like_wachter, n_samples=1000, epsilon=EPSILON_TEST)
        print("ARAR, eps=0.0, like Wachter: ", proportions.mean())
        proportions = experiment_generate_perturbations_random(model, cfs_wachter, n_samples=1000, epsilon=EPSILON_TEST)
        print("Wachter: ", proportions.mean())



#################################################################################################################
######### model_type:  ann
#################################################################################################################
###### Dataset Name:  adult


100%|██████████| 500/500 [15:44<00:00,  1.89s/it]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  277  /  40325  =  0.006869187848729076
ARAR, eps=0.1, pgd:  277  /  40325  =  0.006869187848729076
ARAR, eps=0.05:  7216  /  40325  =  0.17894606323620582
ARAR, eps=0.05, pgd:  7216  /  40325  =  0.17894606323620582
ARAR, eps=0.0:  15621  /  40325  =  0.3873775573465592
ARAR, eps=0.0, like Wachter:  15621  /  40325  =  0.3873775573465592
Wachter:  203  /  500  =  0.406
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.0
ARAR, eps=0.05, pgd:  1.0 0.0
ARAR, eps=0.0:  1.0 0.0
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9749105
ARAR, eps=0.05, pgd:  0.97506785
ARAR, eps=0.0:  0.55173016
ARAR, eps=0.0, like Wachter:  0.6061

100%|██████████| 500/500 [07:04<00:00,  1.18it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  4  /  800  =  0.005
ARAR, eps=0.1, pgd:  4  /  800  =  0.005
ARAR, eps=0.05:  318  /  800  =  0.3975
ARAR, eps=0.05, pgd:  318  /  800  =  0.3975
ARAR, eps=0.0:  526  /  800  =  0.6575
ARAR, eps=0.0, like Wachter:  526  /  800  =  0.6575
Wachter:  322  /  500  =  0.644
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.0
ARAR, eps=0.05, pgd:  1.0 0.0
ARAR, eps=0.0:  1.0 0.0
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9018931
ARAR, eps=0.05, pgd:  0.9027295
ARAR, eps=0.0:  0.5365095
ARAR, eps=0.0, like Wachter:  0.59002095
Wachter:  0.5917857
###### Dataset Name:  give_me_some_credit
No test factuals classified as the ne

...

### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  317  /  317  =  1.0
ARAR, eps=0.1, pgd:  317  /  317  =  1.0
ARAR, eps=0.05:  317  /  317  =  1.0
ARAR, eps=0.05, pgd:  317  /  317  =  1.0
ARAR, eps=0.0:  317  /  317  =  1.0
ARAR, eps=0.0, like Wachter:  317  /  317  =  1.0
Wachter:  317  /  317  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.0
ARAR, eps=0.05, pgd:  1.0 0.0
ARAR, eps=0.0:  1.0 0.0
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.8952524
ARAR, eps=0.05, pgd:  0.89594644
ARAR, eps=0.0:  0.51942587
ARAR, eps=0.0, like Wachter:  0.5940915
Wachter:  0.59408516
###### Dataset Name:  give_me_some_credit


100%|██████████| 500/500 [00:03<00:00, 129.52it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  909  /  909  =  1.0
ARAR, eps=0.1, pgd:  909  /  909  =  1.0
ARAR, eps=0.05:  909  /  909  =  1.0
ARAR, eps=0.05, pgd:  909  /  909  =  1.0
ARAR, eps=0.0:  909  /  909  =  1.0
ARAR, eps=0.0, like Wachter:  909  /  909  =  1.0
Wachter:  500  /  500  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.0
ARAR, eps=0.05, pgd:  1.0 0.0
ARAR, eps=0.0:  1.0 0.0
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9497096
ARAR, eps=0.05, pgd:  0.9498801
ARAR, eps=0.0:  0.5461155
ARAR, eps=0.0, like Wachter:  0.64025635
Wachter:  0.64366394
###### Dataset Name:  heloc


100%|██████████| 500/500 [00:03<00:00, 141.72it/s]


### Number of counterfactuals found / Total number of factuals
ARAR, eps=0.1:  5094  /  5094  =  1.0
ARAR, eps=0.1, pgd:  5094  /  5094  =  1.0
ARAR, eps=0.05:  5094  /  5094  =  1.0
ARAR, eps=0.05, pgd:  5094  /  5094  =  1.0
ARAR, eps=0.0:  5094  /  5094  =  1.0
ARAR, eps=0.0, like Wachter:  5094  /  5094  =  1.0
Wachter:  500  /  500  =  1.0
### predictions classified as target / predictions classified as target after perturbation with gradient
ARAR, eps=0.1:  1.0 1.0
ARAR, eps=0.1, pgd:  1.0 1.0
ARAR, eps=0.05:  1.0 0.0
ARAR, eps=0.05, pgd:  1.0 0.0
ARAR, eps=0.0:  1.0 0.0
ARAR, eps=0.0, like Wachter:  1.0 0.0
Wachter:  1.0 0.0
### Proportion of classifications to target with random perturbations in epsilon ball: 
ARAR, eps=0.1:  1.0
ARAR, eps=0.1, pgd:  1.0
ARAR, eps=0.05:  0.9932804
ARAR, eps=0.05, pgd:  0.99336797
ARAR, eps=0.0:  0.5646785
ARAR, eps=0.0, like Wachter:  0.78642267
Wachter:  0.782816
